## Problem 1 -- Vehicle Routing

A local bakeshop wants determine the minimum cost plan for picking up its daily supply of milk, butter, and eggs from the four farms that supply the bakeshop.  The distance (in miles) between the bakeshop (B) and each farm, and also between each pair of farms, is given in the table below. The table also gives the volume of milk, butter, and eggs (total, in ft$^3$) to be collected from
each farm each day. (The distance between locations is the same in both directions, so for each pair of locations the distance is only reported once.)

| |B | F1 | F2| F3 | F4 |
|:--|:--|:--|:--|:--|:--|
|B| - | 5 | 12 | 7 | 15|
|F1| - | - | 4 | 10 | 7 |
|F2| - | - | - | 14 | 20|
|F3| - | - | - | - |8 |
| | F1 | F2 | F3 | F4 |
|Supply (ft$^3$)|  7 | 2 | 6 | 3 |

The bakeshop has one truck that can carry at most 10 ft$^3$ of supplies at a time. Because of the size limit, the
truck will need to make multiple trips each day to collect the supplies from all the farms. Each trip may pick up supplies from one or more farms, provided the total collected in the trip does not exceed the truck limit. 

Formulate an integer program to help the bakeshop assign farms to the trips so that the total number of trips required every day is minimized  (Hint: model the problem as a set covering problem. The first step will be to list all possible routes a truck can take.)

In [39]:
using JuMP, Cbc, NamedArrays

# Define farms and the bakeshop
locations = ["B", "F1", "F2", "F3", "F4"]
N = length(locations)


dist = NamedArray([0 5 12 7 15
                   5 0 4 10 7 
                  12 4 0 14 20
                  7 10 14 0 8
                  15 7 20 8 0], (locations, locations), ("location", "location")) 

supply = [0, 7, 2, 6, 3]

A = [1 1 0 0
     1 0 0 1
     0 1 1 0
     0 1 0 1
     0 0 1 1]


m = Model(Cbc.Optimizer)

@variable(m, x[1:N], Bin)

@objective(m, Min, sum(dist[i, j] * supply[i] * x[i] for i in 1:N, j in 1:N))

@constraint(m, [j in 1:N], sum(supply[i] * x[i] * A[i] for i in 1:N) >= supply[j])

@constraint(m, sum(supply[i] * x[i] for i in 1:N) <= 10)

optimize!(m)

solution = value.(x)

println("Total distance traveled: ", objective_value(m))
    

Total distance traveled: 182.0
Welcome to the CBC MILP Solver 
Version: 2.10.8 
Build Date: Oct 26 2022 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 182 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from 182 to -1.79769e+308
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Clique was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
MixedIntegerRounding2 was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
FlowC

## Problem 2 -- The Magical Baked Goods Machine

Suppose you are in charge of a magical baked goods machine that creates delicious baked goods of many varieties. Every day, the machine creates batches of 5 different baked goods. To produce a batch of a baked good, you must first clean the machine to remove the remnants of the previous batch of bakery treats (e.g., the workflow could be, "clean, make bread, clean, make donuts,..."). The durations of baking each of the 5 items (donuts, scones, cookies, bread, and coffee cake) are 40, 32, 50, 28, and 47 minutes respectively. The cleaning times depend on the item that was previously made in the machine. For example, a long cleaning period is required if bread is made after scones, since the scones leave a sticky residue from the dried fruit they contain that can ruin the bread. The times are given in minutes in the following table. Each pair $(i,j)$ is the cleaning time required if batch $j$ is baked after batch $i$.

|Baked good|Donut|Scone|Cookie|Bread|Coffee Cake|
|:---|:---|:----|:---|:---|:---|
|Donut|0|10|6|15|9|
|Scone|4|0|6|17|8|
|Cookie|10|11|0|20|14|
|Bread|7|15|6|0|2|
|Coffee Cake|5|8|7|7|0|

We'd obviously like to spend as little time as possible baking and cleaning. What order should we produce the 5 bakery items in? How long do we spend baking and cleaning each day? The order is the same every day, so the cleaning time between the last batch of one week and the first of the following week needs to be accounted for in the total duration of cleaning.

Solve this problem as a TSP. You may use either MTZ reformulation or subtour elimination (or both, for fun!).

In [7]:
using JuMP
using Cbc

items = ["Donut", "Scone", "Cookie", "Bread", "Coffee Cake"]
n_items = length(items)

t = [40, 32, 50, 28, 47]  ## baking time
c = [0  10   6  15   9;
     4   0   6  17   8;
    10  11   0  20  14;
     7  15   6   0   2;
     5   8   7   7   0] ## cleaning times

# Create a model
m = Model(Cbc.Optimizer)


@variable(m, x[1:n_items, 1:n_items], Bin)

@variable(m, u[1:n_items] >= 0)

@constraint(m, [i in 1:n_items], sum(x[i, j] for j in 1:n_items if i != j) == 1)

@constraint(m, [j in 1:n_items], sum(x[i, j] for i in 1:n_items if i != j) == 1)


b = 9000
@constraint(m, [i in 2:n_items, j in 2:n_items], u[i] - u[j] + b * x[i, j] <= b - 1)

@constraint(m, [i in 2:n_items, j in 2:n_items], u[i] - u[j] + (b - t[i]) * x[i, j] + c[i, j] <= b - minimum(t))

@constraint(m, u[1] == 0)

    
@objective(m, Min, sum((t[i] + c[i, j]) * x[i, j] for i in 1:n_items, j in 1:n_items))

optimize!(m)

total_time = objective_value(m)

println("Optimal order: ", opt_order)
println("Total time spent: ", total_time)

Optimal order: Any["Donut", "Scone", "Cookie", "Bread", "Coffee Cake"]
Total time spent: 234.0
Welcome to the CBC MILP Solver 
Version: 2.10.8 
Build Date: Oct 26 2022 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 227.002 - 0.00 seconds
Cgl0004I processed model has 34 rows, 24 columns (20 integer (20 of which binary)) and 112 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 4 integers unsatisfied sum - 0.00133749
Cbc0038I Pass   1: suminf.    0.00133 (4) obj. 227.003 iterations 9
Cbc0038I Pass   2: suminf.    0.00133 (4) obj. 227.003 iterations 0
Cbc0038I Pass   3: suminf.    0.00000 (0) obj. 236 iterations 10
Cbc0038I Solution found of 236
Cbc0038I Relaxing continuous gives 236
Cbc0038I Before mini branch and bound, 9 integers at bound fixed and 1 continuous
Cbc0038I Full problem 34 rows 24 columns, reduced to 10 rows 5 columns
Cbc0038I Mini branch and bound did not improve solution (0.00 seconds)
Cbc0